In [2]:
%pip install openai graphrag pandas requests python-dotenv
from IPython.display import clear_output ; clear_output()

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

is_azure = (
  os.getenv("AZURE_OPENAI_ENDPOINT", default="") != "" and
  os.getenv("OPENAI_API_KEY", default="") == ""
)

if is_azure:
  AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
  AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
  GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")
  TEXT_EMBEDDING_3_LARGE_MODEL_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_MODEL_NAME", default="text-embedding-3-large")
  AZURE_OPENAI_API_VERSION = "2024-05-01-preview"
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [15]:
import requests
import re

wiki_pages = {}

wiki_pages['French_Revolution'] = requests.get("https://en.wikipedia.org/wiki/French_Revolution?action=raw").text
wiki_links = re.findall(r'\[\[(.*?)\]\]', wiki_pages['French_Revolution'])
for link in wiki_links:
  link = link.replace(' ', '_').split('|')[0].split('#')[0]
  if link not in wiki_pages and not link.startswith("File:") and not link.startswith("Image:"):
    url = f"https://en.wikipedia.org/wiki/{link}?action=raw"
    response = requests.get(url)
    if response.status_code == 200:
      wiki_pages[link] = response.text
      print(link) # DEBUG

Atlantic_Revolutions
Storming_of_the_Bastille
Kingdom_of_France
Proclamation_of_the_abolition_of_the_monarchy
Ancien_régime
Constitutional_Cabinet_of_Louis_XVI
French_First_Republic
Reign_of_Terror
execution_of_Louis_XVI
French_Revolutionary_Wars
French_Consulate
France
Estates_General_of_1789
coup_of_18_Brumaire
liberal_democracy
Causes_of_the_French_Revolution
Ancien_Régime
convocation
National_Assembly_(French_Revolution)
Abolition_of_feudalism_in_France
Catholic_Church_in_France
Declaration_of_the_Rights_of_Man_and_of_the_Citizen
economic_depression
insurrection_of_10_August_1792
Insurrection_of_31_May_–_2_June_1793
National_Convention
Committee_of_Public_Safety
Thermidorian_Reaction
French_Directory
Coup_of_18_Brumaire
Napoleon_Bonaparte
elite
Louis_XVI
Parlement
Age_of_Enlightenment
American_Revolution
Affair_of_the_Diamond_Necklace
Farm_(revenue_leasing)
parlement
Anglo-French_War_(1778–1783)
Charles_Alexandre_de_Calonne
Assembly_of_Notables
Étienne_Charles_de_Loménie_de_Brienne

In [16]:
len(wiki_pages)

397